In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime

In [2]:
Online_Retail="data/raw/online_retail_II.xlsx"

In [3]:
# Database Configuration
DB_USER=os.getenv("DB_USER","root")
DB_PWD=os.getenv("DB_PWD","root")
DB_HOST=os.getenv("DB_HOST","localhost")
DB_PORT=os.getenv("DB_PORT","3306")
DB_NAME=os.getenv("DB_NAME","customer_segmentationdb")

In [4]:
# SQLAlchemy connection string for MySQL (using PyMySQL driver)
DATABASE_URL=(f"mysql+pymysql://{DB_USER}:{DB_PWD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [5]:
engine=create_engine(DATABASE_URL,connect_args={"charset":"utf8mb4"})

In [6]:
df_2010=pd.read_excel(Online_Retail,sheet_name="Year 2009-2010")
df_2011=pd.read_excel(Online_Retail,sheet_name="Year 2010-2011")
df=pd.concat([df_2010,df_2011],ignore_index=False)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1067371 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 73.3+ MB


In [8]:
df.describe()

,Quantity,InvoiceDate,Price,Customer ID
count,1.067371e+06,1067371,1.067371e+06,824364.000000
mean,9.938898e+00,2011-01-02 21:13:55.394028544,4.649388e+00,15324.638504
min,-8.099500e+04,2009-12-01 07:45:00,-5.359436e+04,12346.000000
25%,1.000000e+00,2010-07-09 09:46:00,1.250000e+00,13975.000000
50%,3.000000e+00,2010-12-07 15:28:00,2.100000e+00,15255.000000
75%,1.000000e+01,2011-07-22 10:23:00,4.150000e+00,16797.000000
max,8.099500e+04,2011-12-09 12:50:00,3.897000e+04,18287.000000
std,1.727058e+02,NaN,1.235531e+02,1697.464450


In [9]:
# Normalize column names
df.columns=(
        df.columns
          .str.strip()
          .str.replace(" ","_")
          .str.replace(r"[()]","",regex=True)
    )
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer_ID', 'Country'],
      dtype='object')

In [10]:
# Parsing "InvoiceDate" columns
df["InvoiceDate"]=pd.to_datetime(df["InvoiceDate"],errors="coerce")

In [10]:
# Checking for missing values
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer_ID    243007
Country             0
dtype: int64

In [11]:
# Drop missing values
df=df.dropna(subset=["Customer_ID","Description"])
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer_ID    0
Country        0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 824364 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      824364 non-null  object        
 1   StockCode    824364 non-null  object        
 2   Description  824364 non-null  object        
 3   Quantity     824364 non-null  int64         
 4   InvoiceDate  824364 non-null  datetime64[ns]
 5   Price        824364 non-null  float64       
 6   Customer_ID  824364 non-null  float64       
 7   Country      824364 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 56.6+ MB


In [13]:
# Checking for duplicate values
df.duplicated().sum()

np.int64(26479)

In [14]:
# Removing duplicates
df=df.drop_duplicates()

df.duplicated().sum()

np.int64(0)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 797885 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      797885 non-null  object        
 1   StockCode    797885 non-null  object        
 2   Description  797885 non-null  object        
 3   Quantity     797885 non-null  int64         
 4   InvoiceDate  797885 non-null  datetime64[ns]
 5   Price        797885 non-null  float64       
 6   Customer_ID  797885 non-null  float64       
 7   Country      797885 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 54.8+ MB


In [16]:
# Keeping only non-negative values in the columns "Quantity" and "Price"
df=df[df["Quantity"]>=0]
df=df[df["Price"]>=0]

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 779495 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      779495 non-null  object        
 1   StockCode    779495 non-null  object        
 2   Description  779495 non-null  object        
 3   Quantity     779495 non-null  int64         
 4   InvoiceDate  779495 non-null  datetime64[ns]
 5   Price        779495 non-null  float64       
 6   Customer_ID  779495 non-null  float64       
 7   Country      779495 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 53.5+ MB


In [18]:
# Converting entries in the column "Customer_ID" from float to int
df["Customer_ID"]=df["Customer_ID"].astype(int)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 779495 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      779495 non-null  object        
 1   StockCode    779495 non-null  object        
 2   Description  779495 non-null  object        
 3   Quantity     779495 non-null  int64         
 4   InvoiceDate  779495 non-null  datetime64[ns]
 5   Price        779495 non-null  float64       
 6   Customer_ID  779495 non-null  int64         
 7   Country      779495 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(4)
memory usage: 53.5+ MB


In [19]:
df.head(40)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085,United Kingdom
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085,United Kingdom
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085,United Kingdom
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085,United Kingdom


In [20]:
# Saving cleaned and processed data into a CSV file
output_csv=os.path.join("data/processed","processed_online_retail_II.csv")
df.to_csv(output_csv,date_format='%Y-%m-%d %H:%M:%S',index=False)
print(f"Saved processed online_retail_II to:{output_csv}")

Saved processed online_retail_II to:data/processed\processed_online_retail_II.csv


In [21]:
# Loading the processed data to MySQL
df.to_sql(
        name="online_retail_ii",
        con=engine,
        if_exists="replace",
        index=False,
        method="multi",
        chunksize=5000
    )
print("Loaded data into MySQL table `online_retail_ii`\n")

Loaded data into MySQL table `online_retail_ii`

